In [1]:
import pandas as pd

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# 1. 데이터 준비
df = pd.read_csv('C:/Users/user/Downloads/temp_ngram_div20.csv')
# 10000개 샘플 데이터
df = df.sample(n=10000)
df = df.reset_index(drop = True)
word_list = df.values.tolist()

x_data = [i[0] for i in word_list]
y_data = [i[1] for i in word_list]

# 2. 특성 추출 (형태소 분석 및 n-그램)
# mecab = Mecab()
# tagger = mecab.pos
# mecab_tokenize = lambda text: [token for token, pos in tagger(text) if pos == "NNG" or pos == "VA" or pos == "VAX" or pos == "MAG" or pos == "VV"]
vectorizer = CountVectorizer(min_df=15) 
vectorizer.fit(x_data)
x_data = vectorizer.transform(x_data)
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.1, random_state=10, stratify=y_data)

# 3. 나이브 베이즈 모델 학습
clf = MultinomialNB()
clf.fit(x_train, y_train)

# 4. 클래스별 확률 데이터 프레임 생성
proba_array = clf.predict_proba(x_data)
proba_df = pd.DataFrame(proba_array, columns=[f'prob_{cls}' for cls in clf.classes_])
df_with_proba = pd.concat([df, proba_df], axis = 1)

# 5. 클래스별 확률 및 모델 정확도 출력
y_pred = clf.predict(x_test)
print(classification_report(y_test, y_pred))

# 6. 모델 과적합 출력
print(clf.score(x_train, y_train))
print(clf.score(x_test, y_test)) 

              precision    recall  f1-score   support

           0       0.42      0.66      0.51       413
           1       0.28      0.11      0.16       249
           2       0.35      0.26      0.30       338

    accuracy                           0.39      1000
   macro avg       0.35      0.34      0.32      1000
weighted avg       0.36      0.39      0.35      1000

0.45944444444444443
0.388


In [9]:
df_with_proba

,text,labels,prob_0,prob_1,prob_2
0,"('BBB', 'SL')",0,0.366234,0.311012,0.322754
1,"('금리', 'NNG')",2,0.427864,0.223228,0.348908
2,"('Y', 'SL');('Y', 'SL');('Y', 'SL');('Y', 'SL')",2,0.383334,0.317280,0.299386
3,"('밟', 'VV')",1,0.427234,0.221751,0.351015
4,"('후보', 'NNG')",1,0.415811,0.245303,0.338886
...,...,...,...,...,...
9995,"('EPFR', 'SL');('표', 'NNG');('글로벌', 'NNG');('b...",0,0.408233,0.390857,0.200910
9996,"('유럽', 'NNG');('예외', 'NNG')",0,0.522308,0.283032,0.194660
9997,"('과거', 'NNG');('순환적', 'VAX');('금리', 'NNG');('상...",0,0.501371,0.206980,0.291649
9998,"('경제', 'NNG');('지표', 'NNG');('점차', 'MAG');('환율...",0,0.431824,0.190295,0.377881


In [10]:
#score열 생성
df_with_proba['score'] = df_with_proba['prob_2']/df_with_proba['prob_0']

In [11]:
df_with_proba.shape

(10000, 6)

In [12]:
df_with_proba

,text,labels,prob_0,prob_1,prob_2,score
0,"('BBB', 'SL')",0,0.366234,0.311012,0.322754,0.881279
1,"('금리', 'NNG')",2,0.427864,0.223228,0.348908,0.815465
2,"('Y', 'SL');('Y', 'SL');('Y', 'SL');('Y', 'SL')",2,0.383334,0.317280,0.299386,0.781007
3,"('밟', 'VV')",1,0.427234,0.221751,0.351015,0.821599
4,"('후보', 'NNG')",1,0.415811,0.245303,0.338886,0.815002
...,...,...,...,...,...,...
9995,"('EPFR', 'SL');('표', 'NNG');('글로벌', 'NNG');('b...",0,0.408233,0.390857,0.200910,0.492144
9996,"('유럽', 'NNG');('예외', 'NNG')",0,0.522308,0.283032,0.194660,0.372692
9997,"('과거', 'NNG');('순환적', 'VAX');('금리', 'NNG');('상...",0,0.501371,0.206980,0.291649,0.581702
9998,"('경제', 'NNG');('지표', 'NNG');('점차', 'MAG');('환율...",0,0.431824,0.190295,0.377881,0.875082


In [13]:
#중복값 제거
df = df_with_proba.drop_duplicates()

In [14]:
#소수점 2자리수만 남기기
df['score'] = df['score'].round(2)

C:\Users\user\AppData\Local\Temp\ipykernel_13000\2244284570.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score'] = df['score'].round(2)


In [15]:
df.shape

(9159, 6)

In [16]:
#매파/비둘기파에 해당하는 polarity 데이터만 추출
df = df[(df['score'] > 1.3) | (df['score'] < 0.77)]

#매파/비둘기파 라벨링
df['polarity'] = df['score'].apply(lambda x: 'hawkish' if x > 1.3 else 'dovish')

In [17]:
df

,text,labels,prob_0,prob_1,prob_2,score,polarity
7,"('기물', 'NNG');('중심', 'NNG');('금리', 'NNG');('하락...",2,0.568190,0.173731,0.258080,0.45,dovish
9,"('감안', 'NNG');('경기', 'NNG');('있', 'VV');('하방',...",1,0.432815,0.322529,0.244656,0.57,dovish
10,"('국내', 'NNG');('금리', 'NNG');('CD', 'NNG');('통안...",0,0.202708,0.406259,0.391033,1.93,hawkish
11,"('통하', 'VV');('가격변수', 'NNG');('관계', 'NNG');('변...",2,0.611567,0.138809,0.249624,0.41,dovish
19,"('불안요인', 'NNG');('불구', 'NNG')",0,0.555776,0.243368,0.200856,0.36,dovish
...,...,...,...,...,...,...,...
9989,"('방어', 'NNG');('백금', 'NNG');('수입관세', 'NNG');('...",1,0.263429,0.250659,0.485912,1.84,hawkish
9995,"('EPFR', 'SL');('표', 'NNG');('글로벌', 'NNG');('b...",0,0.408233,0.390857,0.200910,0.49,dovish
9996,"('유럽', 'NNG');('예외', 'NNG')",0,0.522308,0.283032,0.194660,0.37,dovish
9997,"('과거', 'NNG');('순환적', 'VAX');('금리', 'NNG');('상...",0,0.501371,0.206980,0.291649,0.58,dovish


In [18]:
df =df[['text', 'polarity']]

In [19]:
df.head()

,text,polarity
7,"('기물', 'NNG');('중심', 'NNG');('금리', 'NNG');('하락...",dovish
9,"('감안', 'NNG');('경기', 'NNG');('있', 'VV');('하방',...",dovish
10,"('국내', 'NNG');('금리', 'NNG');('CD', 'NNG');('통안...",hawkish
11,"('통하', 'VV');('가격변수', 'NNG');('관계', 'NNG');('변...",dovish
19,"('불안요인', 'NNG');('불구', 'NNG')",dovish


In [20]:
#백업 데이터프레임
save =df.copy()

In [21]:
dic = df.set_index('text')['polarity'].to_dict()

In [22]:
# dic.keys().replace(';', ',')
new_key = []

for i in list(dic.keys()):
    i = i.replace(';', ', ')
    new_key.append(i)

old_key = list(dic.keys())

new_dict = {}
for new_key, old_key in zip(new_key, old_key):
    new_dict[new_key] = dic[old_key]
new_dict

{"('지난', 'VV'), ('간', 'NNG'), ('일본', 'NNG'), ('국채', 'NNG')": 'dovish',
 "('월간', 'NNG'), ('채권투자', 'NNG'), ('전망', 'NNG'), ('강세', 'NNG'), ('지속', 'NNG')": 'hawkish',
 "('상위', 'NNG'), ('종목', 'NNG'), ('만기수익률', 'NNG')": 'dovish',
 "('기조', 'NNG'), ('완화', 'NNG')": 'dovish',
 "('결정', 'NNG'), ('최근', 'NNG'), ('물가', 'NNG'), ('불안', 'NNG'), ('주', 'NNG')": 'dovish',
 "('금리', 'NNG'), ('bp', 'SL')": 'dovish',
 "('외국인', 'NNG'), ('순매도', 'NNG'), ('상위', 'NNG')": 'dovish',
 "('지수', 'NNG'), ('소비자', 'NNG'), ('물가', 'NNG')": 'dovish',
 "('기간', 'NNG'), ('변동', 'NNG'), ('기간', 'NNG')": 'dovish',
 "('지표채권', 'NNG'), ('육성', 'NNG'), ('일환', 'NNG'), ('단기', 'NNG'), ('국채', 'NNG')": 'dovish',
 "('실업률', 'NNG')": 'dovish',
 "('공모주', 'NNG'), ('하이일드', 'NNG'), ('펀드설정액', 'NNG'), ('꾸준히', 'MAG')": 'dovish',
 "('합계', 'NNG'), ('가중평균', 'NNG'), ('잔존만기', 'NNG'), ('수량', 'NNG')": 'dovish',
 "('AA', 'SL'), ('회사채', 'NNG'), ('A', 'SL'), ('회사채', 'NNG'), ('A', 'SL')": 'dovish',
 "('AA', 'SL'), ('개발공사', 'NNG'), ('AA', 'SL'), ('도시', 'NNG')": 'dov

In [23]:
new_dict.keys()

dict_keys(["('지난', 'VV'), ('간', 'NNG'), ('일본', 'NNG'), ('국채', 'NNG')", "('월간', 'NNG'), ('채권투자', 'NNG'), ('전망', 'NNG'), ('강세', 'NNG'), ('지속', 'NNG')", "('상위', 'NNG'), ('종목', 'NNG'), ('만기수익률', 'NNG')", "('기조', 'NNG'), ('완화', 'NNG')", "('결정', 'NNG'), ('최근', 'NNG'), ('물가', 'NNG'), ('불안', 'NNG'), ('주', 'NNG')", "('금리', 'NNG'), ('bp', 'SL')", "('외국인', 'NNG'), ('순매도', 'NNG'), ('상위', 'NNG')", "('지수', 'NNG'), ('소비자', 'NNG'), ('물가', 'NNG')", "('기간', 'NNG'), ('변동', 'NNG'), ('기간', 'NNG')", "('지표채권', 'NNG'), ('육성', 'NNG'), ('일환', 'NNG'), ('단기', 'NNG'), ('국채', 'NNG')", "('실업률', 'NNG')", "('공모주', 'NNG'), ('하이일드', 'NNG'), ('펀드설정액', 'NNG'), ('꾸준히', 'MAG')", "('합계', 'NNG'), ('가중평균', 'NNG'), ('잔존만기', 'NNG'), ('수량', 'NNG')", "('AA', 'SL'), ('회사채', 'NNG'), ('A', 'SL'), ('회사채', 'NNG'), ('A', 'SL')", "('AA', 'SL'), ('개발공사', 'NNG'), ('AA', 'SL'), ('도시', 'NNG')", "('입찰', 'NNG')", "('통안채', 'NNG'), ('매수', 'NNG'), ('통안채', 'NNG'), ('중', 'NNG')", "('일부', 'NNG'), ('산업', 'NNG'), ('종목', 'NNG'), ('산업', 'NNG')", "('다음',

In [25]:
#tokenize_text불러오기

import pandas as pd
data = pd.read_csv('C:/Users/user/Downloads/tokenize_text.csv',index_col=0)

In [26]:
data.head()

,text,labels
0,"('Daewoo', 'SL'), ('Bond', 'SL'), ('Brief', 'S...",0
1,"('Daewoo', 'SL'), ('Bond', 'SL'), ('Brief', 'S...",0
2,"('Daewoo', 'SL'), ('Bond', 'SL'), ('Brief', 'S...",0
3,"('Daewoo', 'SL'), ('Bond', 'SL'), ('Brief', 'S...",0
4,"('Daewoo', 'SL'), ('Bond', 'SL'), ('Brief', 'S...",0


In [27]:
txt = data.copy()

In [28]:
txt.shape

(304653, 2)

In [31]:
txt.head()

,text,labels
0,"('Daewoo', 'SL'), ('Bond', 'SL'), ('Brief', 'S...",0
1,"('Daewoo', 'SL'), ('Bond', 'SL'), ('Brief', 'S...",0
2,"('Daewoo', 'SL'), ('Bond', 'SL'), ('Brief', 'S...",0
3,"('Daewoo', 'SL'), ('Bond', 'SL'), ('Brief', 'S...",0
4,"('Daewoo', 'SL'), ('Bond', 'SL'), ('Brief', 'S...",0


In [32]:
#sample {key:value} 추가
# new_dict[('Bond', 'SL')] = 'dovish'
# new_dict[('하락', 'NNG')] = 'hawkish'
# new_dict[('Daewoo', 'SL')] ='dovish'

In [33]:
#각 text 토큰마다 사전 조회 후 사전에 해당 단어 있을 경우 count+1
import ast

def count_tags(row):
    hawkish_count = 0
    dovish_count = 0

    # 문자열로 저장된 튜플 리스트를 실제 리스트로 변환
    tuples = ast.literal_eval(row['text'])

    for t in tuples:
        tag = new_dict.get(t)
        if tag == 'hawkish':
            hawkish_count += 1
        elif tag == 'dovish':
            dovish_count += 1

    return pd.Series([hawkish_count, dovish_count])

txt[['hawkish_count', 'dovish_count']] = txt.apply(count_tags, axis=1)


In [50]:
import pandas as pd
# 데이터프레임 전체 출력
# pd.set_option('display.max_rows', 10)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', 10)
 

In [53]:
txt['tone'] = (txt['hawkish_count'] - txt['dovish_count'])/(txt['hawkish_count']+txt['dovish_count'])

In [54]:
txt.columns

Index(['text',
       'labels',
       'hawkish_count',
       'dovish_count',
       'tone'],
      dtype='object')

In [58]:
txt.shape

(1000, 5)

In [60]:
txt.head()

text  \
0  ('Daewoo', 'SL'), ('Bond', 'SL'), ('Brief', 'SL'), ('Daewoo', 'SL'), ('채권', 'NNG'), ('ikyoon', 'SL'), ('bestez', 'SL'), ('com', 'SL'), ('Fri', 'SL'), ('채권', 'NNG'), ('시황', 'NNG'), ('국고채', 'NNG'), ('금리', 'NNG'), ('bp', 'SL'), ('하락', 'NNG'), ('금리', 'NNG'), ('bp', 'SL'), ('하락', 'NNG'), ('마감', 'NNG'), ('장', 'NNG'), ('초반', 'NNG'), ('외국인', 'NNG'), ('선물', 'NNG'), ('계약', 'NNG'), ('이상', 'NNG'), ('순매수', 'NNG'), ('드레싱', 'NNG'), ('채권시장', 'NNG'), ('강세', 'NNG'), ('출발', 'NNG'), ('이후', 'NNG'), ('산업', 'NNG'), ('생산', 'NNG'), ('지표', 'NNG'), ('기대', 'NNG'), ('금리', 'NNG'), ('낙폭', 'NNG'), ('서서히', 'MAG'), ('확대', 'NNG'), ('시작', 'NNG'), ('산업', 'NNG'), ('생산', 'NNG'), ('지표', 'NNG'), ('예상', 'NNG'), ('크', 'VA'), ('하회', 'NNG'), ('전년비', 'NNG'), ('기록', 'NNG'), ('IMF', 'SL'), ('당시', 'NNG'), ('안', 'MAG'), ('좋', 'VA'), ('수치', 'NNG'), ('경기', 'NNG'), ('악화', 'NNG'), ('급속', 'NNG'), ('이루', 'VV'), ('아직', 'MAG'), ('저점', 'NNG'), ('아니', 'VCN'), ('인식', 'NNG'), ('국고채', 'NNG'), ('금리', 'NNG'), ('급락', 'NNG'), ('약세', 'NNG'), ('예상', 'NNG'), ('더', 'MAG'), ('악화', 'NNG'), ('경기', 'NNG'), ('향후', 'MAG'), ('통', 'MAG'), ('위시', 'NNG'), ('금리', 'NNG'), ('하폭', 'NNG'), ('더', 'MAG'), ('확대', 'NNG'), ('있', 'VV'), ('인하', 'NNG'), ('기조', 'NNG'), ('당분간', 'MAG'), ('계속', 'NNG'), ('보', 'VV'), ('한국은행총재', 'NNG'), ('신년사', 'NNG'), ('내년', 'NNG'), ('경기', 'NNG'), ('어렵', 'VA'), ('시사', 'NNG'), ('금융시장', 'NNG'), ('안정', 'NNG'), ('경기', 'NNG'), ('회복', 'NNG'), ('주안점', 'NNG'), ('두', 'VV'), ('정책', 'NNG'), ('운용', 'NNG'), ('밝히', 'VV'), ('채권시장', 'NNG'), ('우호적', 'VAX'), ('환경', 'NNG'), ('조성', 'NNG'), ('당장', 'MAG'), ('일', 'NNG'), ('국고채', 'NNG'), ('수급', 'NNG'), ('상황', 'NNG'), ('보', 'VV'), ('금리', 'NNG'), ('인하', 'NNG'), ('기대', 'NNG'), ('한은', 'NNG'), ('유동성', 'NNG'), ('공급', 'NNG'), ('강세', 'NNG'), ('기조', 'NNG'), ('계속', 'NNG'), ('판단', 'NNG'), ('관심', 'NNG'), ('차트', 'NNG'), ('경제', 'NNG'), ('지표', 'NNG'), ('산업', 'NNG'), ('생산', 'NNG'), ('지표', 'NNG'), ('예상', 'NNG'), ('크', 'VA'), ('하회', 'NNG'), ('경기', 'NNG'), ('우려', 'NNG'), ('크', 'VV'), ('금리', 'NNG'), ('급락', 'NNG'), ('GMAC', 'SL'), ('행지', 'NNG'), ('주', 'NNG'), ('전환', 'NNG'), ('불구', 'NNG'), ('경기', 'NNG'), ('지정학', 'NNG'), ('불안', 'NNG'), ('금리', 'NNG'), ('하락', 'NNG'), ('금리', 'NNG'), ('국고채', 'NNG'), ('채국', 'NNG'), ('고채', 'NNG'), ('CD', 'NNG'), ('통안채', 'NNG'), ('회사채', 'NNG'), ('AA', 'SL'), ('금리', 'NNG'), ('미국', 'NNG'), ('미국', 'NNG'), ('미국', 'NNG'), ('미국', 'NNG'), ('TIPS', 'SL'), ('자료', 'NNG'), ('증권업협회', 'NNG'), ('리서치', 'NNG'), ('EZ', 'SL'), ('DB', 'NNG'), ('자료', 'NNG'), ('Datastream', 'SL'), ('ABS', 'SL'), ('잔액', 'NNG'), ('글로벌', 'NNG'), ('CDO', 'SL'), ('발행', 'NNG'), ('액', 'NNG'), ('오토', 'NNG'), ('리스', 'NNG'), ('카드론', 'NNG'), ('홈', 'NNG'), ('에퀴티', 'NNG'), ('기타', 'NNG'), ('글로벌', 'NNG'), ('CDO', 'SL'), ('발행', 'NNG'), ('액', 'NNG'), ('Q', 'SL'), ('Q', 'SL'), ('Q', 'SL'), ('Q', 'SL'), ('Q', 'SL'), ('Q', 'SL'), ('Q', 'SL'), ('Q', 'SL'), ('Q', 'SL'), ('Q', 'SL'), ('Q', 'SL'), ('자료', 'NNG'), ('Bloomberg', 'SL'), ('자료', 'NNG'), ('Bloomberg', 'SL'), ('Daewoo', 'SL'), ('Bond', 'SL'), ('Brief', 'SL'), ('Daewoo', 'SL'), ('채권', 'NNG'), ('ikyoon', 'SL'), ('bestez', 'SL'), ('com', 'SL'), ('채권시장', 'NNG'), ('주요', 'NNG'), ('지표', 'NNG'), ('종류', 'NNG'), ('CD', 'NNG'), ('CP', 'NNG'), ('통안채', 'NNG'), ('통안채', 'NNG'), ('국고채', 'NNG'), ('국고채', 'NNG'), ('국고채', 'NNG'), ('국고채', 'NNG'), ('회사', 'NNG'), ('회사', 'NNG'), ('US', 'SL'), ('TB', 'SL'), ('m', 'SL'), ('US', 'SL'), ('TN', 'SL'), ('y', 'SL'), ('US', 'SL'), ('TN', 'SL'), ('y', 'SL'), ('US', 'SL'), ('TB', 'SL'), ('y', 'SL'), ('유로', 'NNG'), ('리보', 'NNG'), ('m', 'SL'), ('리보', 'NNG'), ('M', 'SL'), ('코스피', 'NNG'), ('KOSDAQ', 'SL'), ('Dow', 'SL'), ('NASDAQ', 'SL'), ('영국', 'NNG'), ('FTSE', 'NNG'), ('Nikkei', 'SL'), ('항셍지수', 'NNG'), ('상해', 'VV'), ('종합', 'NNG'), ('금리', 'NNG'), ('해외', 'NNG'), ('금리', 'NNG'), ('주가', 'NNG'), ('기관', 'NNG'), ('장외거래', 'NNG'), ('대금', 'NNG'), ('거래대금', 'NNG'), ('순매수', 'NNG'), ('은행', 'NNG'), ('보험', 'NNG'), ('투신', 'NNG'), ('기금', 'NNG'), ('공제', 'NNG'), ('외국인', 'NNG'), ('요구불예금', 'NNG'), ('수신증', 'NNG'), ('잔액', 'NNG'), ('음영', 'NNG'), ('부분', 'NNG'), ('